In [453]:
#Import the following libraries
import numpy as np
import cv2
import os
from PIL import Image
from numpy import linalg

In [454]:
#copying the files in the following path(data) to the list(lst)
data = "C:\\Users\\saich\\Desktop\\Face_Recognition\\YaleFaceDatabase\\"
lst = np.array(os.listdir(data))
images=[]
#Flattening the image and converting them to ndarray
for i in lst:
    img=Image.open('Face_Recognition/YaleFaceDatabase/'+i).convert('L')
    img=np.array(img,'uint8')
    img = img.flatten()
    img=list(img)
    images.append(img)
images = np.array(images)

In [455]:
train_img = []
test_img=[]
# I have used the Yale dataset. It has images of 15 persons 11 images each with different emotions.
#I have taken first 8 images for training and next 3 images for testing for all the 15 persons
for i in range(len(images)):
    if i%11>=8:
        test_img.append(images[i,:])
    else:
        train_img.append(images[i,:])
        # print(train_img[0])
#Converting the list to ndarray
test_img = np.array(test_img)
train_img = np.array(train_img)


In [456]:
#calculating the average image
avgimg = train_img.mean(axis = 0)
#image matrix after subtracting mean image
phi_transpose = np.subtract(train_img,avgimg)
phi = phi_transpose.transpose()
#C is the covariance matrix
c = np.matmul(phi_transpose,phi)

In [457]:
#Calculating the eigen values and eigen vectors of the covariance matrix (A.T)A
eigval,eigvec = linalg.eigh(c)
#Adding all the eigen values
eigval_sum = eigval.sum()
findk = 0
eigval_rev = eigval[::-1]
#eigvec_rev = eigvec[::-1]
for i in range(120):
    findk = (eigval_rev[i])/eigval_sum + findk
    if findk>=0.9:
        k = i
        break
#Let us take the top k eigen vectors of the corresponding eigen values

In [458]:
e=np.array(eigvec).transpose()
eigvec_rev=e[::-1]
k_eigvec = eigvec_rev[:k, :]
#Calculating the eigan vectors of the actual matrix AA.T
real_eigvec_beforenorm = np.dot(phi,k_eigvec.T)
#normalizing the eigen vectors
real_eigvec = np.divide(real_eigvec_beforenorm,linalg.norm(real_eigvec_beforenorm))
train_weights = []
#Calculating the weights of the training set
for vector in phi_transpose:
    train_weights.append(np.dot(real_eigvec.T, vector))
train_weights=np.array(train_weights)

In [459]:
#Normalize the test set by subtracting mean from test set
test_normalized = np.subtract(test_img,avgimg)
test_norm = test_normalized.transpose()
test_weights = np.matmul(real_eigvec.T,test_norm)
#Calculating the error
#Here i have used Euclidian norm as error function
#it has been reported that the Mahalanobis distance performs better:
w=0
for i in range(45):
    test_weight=test_weights[:, i]
    loss=[]
    for j in range(120):
        weight=train_weights[j]
        loss.append(linalg.norm(test_weight-weight, 2))
    min_loss=min(loss)
    min_index=loss.index(min_loss)
    if (min_index//8) == (i//3):
        w=w+1
test_accuracy = w/45
#Accuracy of the test set
print(test_accuracy)

0.9333333333333333
